
<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/notebooks/020_RAG_Architecture_Simulation.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



<a href="https://colab.research.google.com/github/takzen/ai-engineering-handbook/blob/main/20_RAG_Architecture_Simulation.ipynb" target="_parent">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


# 🤖 RAG: Jak rozmawiać z własnymi PDF-ami?

ChatGPT jest świetny, ale ma jedną wadę: **nie zna Twoich sekretów**. Nie wie, jaki jest regulamin urlopowy w Twojej firmie, ani co wczoraj zjadłeś na śniadanie.

Mamy dwa wyjścia:
1.  **Fine-Tuning:** Douczamy model (bardzo drogie i trudne).
2.  **RAG (Retrieval Augmented Generation):** Oszukujemy system.

**Jak działa RAG?**
Zamiast uczyć studenta (model) całej encyklopedii na pamięć, dajemy mu **otwartą książkę** podczas egzaminu.

1.  **Retrieval (Wyszukanie):** Użytkownik zadaje pytanie. My przeszukujemy naszą bazę i znajdujemy odpowiedni fragment tekstu (korzystając z Cosine Similarity).
2.  **Augmentation (Rozszerzenie):** Doklejamy ten fragment do pytania jako "Kontekst".
3.  **Generation (Generowanie):** Wysyłamy całość do LLM.

W tym notatniku zbudujemy ten proces od zera.

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 1. TWOJA PRYWATNA BAZA WIEDZY
# Wyobraź sobie, że to są fragmenty wyciągnięte z firmowych PDF-ów.
knowledge_base = [
    "ID_01: Polityka urlopowa: Każdemu pracownikowi przysługuje 26 dni urlopu rocznie.",
    "ID_02: Godziny pracy: Biuro jest czynne od 8:00 do 16:00, ale piątki są zdalne.",
    "ID_03: Hasło do Wi-Fi: Sieć nazywa się 'Firma_Guest', hasło to 'SuperTajne123'.",
    "ID_04: Ekspres do kawy: Aby zrobić latte, naciśnij dwa razy guzik z ziarnem.",
    "ID_05: Kontakt IT: W razie awarii dzwoń do Michała pod numer 555-000-111."
]

# 2. ZAMIANA NA WEKTORY (TF-IDF)
# Używamy TF-IDF zamiast zwykłego liczenia słów, bo jest mądrzejsze
# (ignoruje słowa typu "i", "w", a skupia się na unikalnych jak "Wi-Fi").
vectorizer = TfidfVectorizer()
base_vectors = vectorizer.fit_transform(knowledge_base)

print(f"Baza wiedzy zindeksowana! Mamy {len(knowledge_base)} dokumentów.")

Baza wiedzy zindeksowana! Mamy 5 dokumentów.


## Krok 1: Retrieval (Wyszukiwarka)

To jest element, który decyduje o inteligencji systemu. Jeśli tutaj zawiedziemy, model dostanie złe dane i zacznie kłamać ("Garbage In, Garbage Out").

Napiszemy funkcję `retrieve_context`, która:
1.  Bierze pytanie użytkownika.
2.  Zamienia je na wektor.
3.  Liczy podobieństwo do każdego dokumentu w bazie.
4.  Zwraca ten najlepszy.

In [4]:
def retrieve_context(user_query, documents, vectorizer, document_vectors):
    # 1. Zamień pytanie na wektor
    query_vec = vectorizer.transform([user_query])
    
    # 2. Policz podobieństwo (Cosine Similarity)
    similarities = cosine_similarity(query_vec, document_vectors).flatten()
    
    # 3. Znajdź indeks najlepszego dokumentu
    best_idx = np.argmax(similarities)
    best_score = similarities[best_idx]
    
    # Zwróć treść dokumentu i jego wynik pewności
    return documents[best_idx], best_score

# TESTUJEMY!
pytania = [
    "Jakie jest hasło do neta?",
    "Ile mam wolnego w roku?",
    "Co zrobić jak komputer nie działa?"
]

print("--- TEST WYSZUKIWARKI (RETRIEVAL) ---")
for p in pytania:
    doc, score = retrieve_context(p, knowledge_base, vectorizer, base_vectors)
    print(f"Pytanie: '{p}'")
    print(f"Znaleziono (Pewność {score:.2f}):\n   -> {doc}")
    print("-" * 30)

--- TEST WYSZUKIWARKI (RETRIEVAL) ---
Pytanie: 'Jakie jest hasło do neta?'
Znaleziono (Pewność 0.42):
   -> ID_03: Hasło do Wi-Fi: Sieć nazywa się 'Firma_Guest', hasło to 'SuperTajne123'.
------------------------------
Pytanie: 'Ile mam wolnego w roku?'
Znaleziono (Pewność 0.00):
   -> ID_01: Polityka urlopowa: Każdemu pracownikowi przysługuje 26 dni urlopu rocznie.
------------------------------
Pytanie: 'Co zrobić jak komputer nie działa?'
Znaleziono (Pewność 0.30):
   -> ID_04: Ekspres do kawy: Aby zrobić latte, naciśnij dwa razy guzik z ziarnem.
------------------------------


## Krok 2: Augmentation & Generation (Konstrukcja Promptu)

Teraz najważniejsza część. Nie mamy tu podpiętego prawdziwego GPT-4 (bo wymaga klucza API), ale **zasymulujemy** to, co dzieje się wewnątrz aplikacji typu ChatPDF.

Musimy stworzyć **Prompt Systemowy**, który łączy:
1.  Rolę AI ("Jesteś pomocnym asystentem").
2.  Znaleziony kontekst ("Tu masz dane: ...").
3.  Pytanie użytkownika.

In [5]:
def generate_rag_prompt(user_query):
    # 1. Najpierw znajdź wiedzę (Retrieval)
    context_text, score = retrieve_context(user_query, knowledge_base, vectorizer, base_vectors)
    
    # Jeśli system nie jest pewny (niski score), nie zmyślajmy
    if score < 0.2:
        return "System: Nie znalazłem informacji w bazie dokumentów. Proszę skontaktować się z HR."
    
    # 2. Sklej wszystko w jeden Prompt dla LLM (Augmentation)
    final_prompt = f"""
    --- ROLA ---
    Jesteś asystentem biurowym. Odpowiadaj TYLKO na podstawie poniższego kontekstu.
    Nie używaj wiedzy z zewnątrz.
    
    --- KONTEKST (TAJNE DANE FIRMOWE) ---
    {context_text}
    
    --- PYTANIE UŻYTKOWNIKA ---
    {user_query}
    """
    
    return final_prompt

# SYMULACJA RAG
pytanie_usera = "Ej, jak się połączyć z wifi?"

prompt = generate_rag_prompt(pytanie_usera)

print("--- TO WYSYŁAMY DO GPT-4 (POD MASKĄ) ---")
print(prompt)

--- TO WYSYŁAMY DO GPT-4 (POD MASKĄ) ---

    --- ROLA ---
    Jesteś asystentem biurowym. Odpowiadaj TYLKO na podstawie poniższego kontekstu.
    Nie używaj wiedzy z zewnątrz.

    --- KONTEKST (TAJNE DANE FIRMOWE) ---
    ID_03: Hasło do Wi-Fi: Sieć nazywa się 'Firma_Guest', hasło to 'SuperTajne123'.

    --- PYTANIE UŻYTKOWNIKA ---
    Ej, jak się połączyć z wifi?
    


## 🧠 Podsumowanie: Iluzja Inteligencji

Zobacz, co się stało.
Użytkownik zapytał o "wifi". W bazie nie było słowa "wifi" (było "Wi-Fi" lub "sieć"), ale wektoryzator to wyłapał (lub wyłapałby lepszy model Embeddings).

System RAG działa jak sufler w teatrze:
1.  Aktor (LLM) zapomniał tekstu.
2.  Sufler (Retrieval) szybko kartkuje scenariusz i podpowiada **tylko to jedno zdanie**, które jest teraz potrzebne.
3.  Aktor wypowiada je płynnie.

**Wniosek:**
Jako AI Engineer, Twoim głównym zadaniem jest budowanie **dobrego Suflera**.
Bo nawet GPT-5 nie odpowie poprawnie na pytanie o Twoją firmę, jeśli Sufler poda mu złą stronę scenariusza (zły kontekst).